# 4. Spark Architect and Performance

### Clusters
- Example with a Cluster has 1 Driver, 6 Workers. Each Worker has 1 Executor and 2 Cores.
  - Driver: brain of cluster which allocate tasks and data to worker nodes
  - Worker: receives tasks and data, performs and return result to Deiver

## Transformation

### Narrow vs Wide Transformation
- Narrow Transformation: 1-1 Partition
  - select, filter, cast, union
  - Start with 1 memory partition, do transformations and data stay within the **same** memory partition
- Wide Transformation: Causes Shuffle/Exchange
  - distinct, groupBy, sort, join
  - Redistribute data and then create **new** memory partitions
  - Redistibuting or re-partitioning data so the data is grouped differently across partitions
    - Based on data size we may need to decrease/increase the number of Shuffle partitions via ```spark.sql.shuffle.patitions```

![Narrow vs Wide Transformation](./images/Narrow_And_Wide_Transformation.png)

#### Process of Narrow Transformation
Example: Filter color != "brown". We have 12-16 memory partitions of data
- Step 1: Driver puts data files into 12-16 equal sized parition
- Step 2: Driver allocates 12 partitions to 12 Cores, each Core gets 1 partition --> Opps we still have 4 partitions left
- Step 3: 4 Cores finish early and return result to Driver. The other Cores are still processing
- Step 4: Driver allocates 4 partitions for another iteration to the 4 Cores. The other Cores finish
- Step 5: 4 Cores finish the 2nd iteration and return resul to Driver
- Step 6: Driver collects the result and delivers to the client

![Process of Narrow Transformation](./images/Narrow_Transformation_Example.png)

#### Process of Wide Transformation
Example: Count total rows for each color. We have **19.5 MiB** data size, with 6 initial partitions
- Stage 1: Local cCount
  - Step 1: Driver allocates 6 partitions to 6 Cores, each Core gets 1 partition
  - Step 2: 6 Cores finish early and **write** the result into Disk in dictionary key:value, so the file is only **568B**. For example:
    - Core 2: Red:3, Blue:5, Yellow:7
    - Core 3: Red:4, Blue:4, Yellow:8
    - ...
    - Core 12: Red:7, Blue:5, Yellow:5
- Stage 2: Global Count
  - Step 1: Driver allocates Core 7 to read the counts and do the "Global Count", then send the result
  - Step 2: Core 7 **read** and sum the Local Counts and return the result to Driver
  - Step 3: Driver collects the result and delivers to the client

![Process of Wide Transformation](./images/Wide_Transformation_Example.png)

## Performance and Query Optimization
There are 5 plans:

**Input: Query, no matter programming language -->**
1. Unresolved Logical Plan
    - Drive reviews and confirms the **```schema correct?```**

2. Analyzed Logical Plan
    - Drive looks at **[Metadata Catalog] for ANALYSIS** and **```add data types```** to the columns

3. Optimized Logical Plan
    - Driver looks at **[Catalyst Catalog] for LOGICAL OPTIMIZATION** and check to apply number of rules-based (Filter, Join, etc.) to determine whether to **```move Filter before Join?```**

4. Physical Plan
    - Driver comes up with several ways (plans) to address the query **for PHYSICAL PLANNING** to determine **```which Join strategy to use, Data Skipping, Predicate Pushdown?```**

5. Selected Physical Plan
    - Driver puts ways in **[Cost Model] for WHOLE-STAGE CODE GENERATION** to see which way (plan) needs lowest cost = the best
    - The plan would be **```Java bytecode and sent to Executors```**

**--> Output: RDD, no matter dataframe, sql table or view**

![Query Optimization](./images/Query_Optimization.png)

### Explain
![Explain](./images/Explain.png)

For each step, read from bottom to top

**Step 1: Parsed Logical Plan**
- It is shown as the script is wrote: Inner Join then Filter
- Check the schema, e.g. lastname, firstname. dept... (no data types)

**Step 2: Analyzed Logical Plan**
- Add data type to each column of schema in step 1
- Add CAST to ```dept``` to double as it is string

It does not display all possible plans, but a optimized plan

**Step 3: Optimized Logical Plan**
- Move the Filter before Join, to get less rows to join

**Step 4: Physical PLan**
- Pick the best plan: Filter before Join and choose "Sort Merge Join" instead of "Inner Join"


### Cost Based Optimization (CBO): Smaller Tables Join
![Cost Based Optimization (CBO)](./images/CBO.png)

- Enable configurations: CBO, join reorder
- In script, (2) we join **```large```** table to **```medium```** table, then (1) we join **```small```** table
- In CBO step, it chooses to:
  - (1) get **```small```** table join with **```medium```** table then join with **```large```** table last
  - Afterward, (2) it does shuffle/exchange




### Adaptive Query Execution (AQE)
![Adaptive Query Execution (AQE)](./images/AQE.png)
- After RDD created, Spark will look at Statistics and shuffle count to see how big they are. Then, it will turn back to Analyzed Logical Plan to see whether it can fine-tune the number of shuffle


![With And Without AQE](./images/With_Without_AQE.png)
- **Without AQE:** we (1) start with 4 memory partitions and (2) end up with 200 memory partitions. **_But why does it need to shuffle 200 partitions for only 568B?_**, which mean 2B foreach partition --> too small
- **With AQE:** we (1) start with 4 memory partitions and (2) end up with 1 partition, and much faster, even without parallism


![Number of Jobs of AQE](./images/AQE_Number_Of_Jobs.png)
- Number of jobs **increase** from 1 to 3 (job #3, #4, #5)
  - Job #3 filters at WHERE clause to reduce size.
  - Job #4 with AQE, needs 1 Shuffle
- Hence, it **has cost overhead** with it, but it has benefit in long run

### Predicate Pushdown on RDDs
- Predicate Pushdown is when the data source actively **limits the number** of rows returned to Spark reader vi SELECT/WHERE/FILTER
- Predicate Pushdown filters the data in the database query,
  - Reducing the number of entries retrieved from the source database and 
  - Improving query performance
  - By default, the Spark Dataset API will automatically push down valid WHERE clauses to the database
- **Cast** function cannot be Pushed down

![Predicate Pushdown](./images/Predicate_Pushdown.png)

### Caching - Best Practices
- Don't cach unless you're sure a DataFrame will be **used multiple times**
  - e.g. EDA (Exploratory Data Analysis), ML traning dataset
- Omit unneeded columns to reduce the storage footprint
- After calling `.cache()` which is **lazy transformation**, ensure all partitions are accessed with **Action**
  - e.g. `count()` - put RDD into Cache
- Manually evict cache when not needed
  - e.g. `unpersist()` - remove RDD from Cache

## Memory Partitioning

### Guidelines
- Error on the side of **too many small** than to few large Memory Partitions. If so large memory, then the core does not have enough memory, leading 2 possible consequences:
  - Spill to disk, waiting for more RAM, then bring it back
  - OOM: out of memory error
- Sweet spot initial size: **128MB and 1GB**
- Calculate the size of Shuffle partitions by dividing Shuffle stage input (4TB) by the target partition size (200MB).
  - e.g. 4TB / 200MB = 20,000 Shuffle Partition count
  - By default, it is 200 `spark.conf.get("spark.sql.shuffle.partitions")`
- Can manually set number of Shuffle Partitions on case-by-case basis
  - `spark.conf.set("spark.sql.shuffle.partitions", "20000")`
  - This setting is Local for **1 session** only.

![200_Default_Partitions](./images/200_Default_Partitions.png)
- Example: 
  - It starts with 8 partitions, then spawns 200 shuffle partitions
  - But there are only 42KB (~no thing) to write
  - Even some of tasks which means memory partitions reside has 0 bytes (blank)
  - It looks like AEQ turned off --> turn it on

### Cores in Cluster
- Initially, the Driver determines the number of Memory Partitions and its size. It decides based on:
  - Number of Cores in Cluster. 
  - More Cores, more Patitions.
- Get to number of Cores by 2 ways:
  - `sc.defaultParallelism` or `spark.sparkContext.defaultParallelism`
  - Spark UI -> Cores
  
  ![Spark UI](./images/Spark_UI_Cores.png)


### No. of Memory Partition for DataFrame
- If Memory Partitions are sized too large (> 1GB), we can manually change in No. of Partitions (to higher number) to get them into a more reasonable size rang (**128MB** to **1GB**)
- AQE can resolve some Partitions issues
  - e.g. for small dataset, AQE won't create default 200 Shuffle Partitions, but rather a far lower number
- We need to **convert DataFrame into RDD** to get number of Partitions used for the DataFrame
  - `df.rdd.getNumPartitions()`

### Re-Partition a DataFrame
There are 2 ways:
1. **`coalesce(int)`:**
    - Returns new DF with exactly N partitions when N < current No. of Partitions
    - **Narrow** transformation
    - Pros:
        - Retain sort order
        - No shuffle
    - Cons:
        - Only decrease No. of Partitions
        - Unevenly balanced partition sizes

2. **`repartition(int, [col])`:**
    - Return new DF with exactly N partitions
    - **Wide** transformation
    - Pros:
        - Evenly balanced partition sizes
        - Both increase/decrease No. of Partition
    - Cons:
        - Not retain sort order
        - Require Shuffle

**Notes:**
  - More No. of Partition, less size
  - Less No. of Paritions, more size